In [2]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import numpy as np 

In [3]:
from multiprocessing import Pool


In [8]:
def parsing_reviews (film_name): #возвращает массив типа [название, отзывы, оценка]
    
    film_name_new = film_name.replace(' ','+')
    url = 'https://www.imdb.com/find?ref_=nv_sr_fn&q='+ film_name_new + '&s=all'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    flag = 0
    if (not soup.findAll("div", attrs={"class": ["findNoResults"]})): #если фильм существует
        if(soup.findAll("h3", attrs={"class": ["findSectionHeader"]})):
            if (soup.findAll("h3", attrs={"class": ["findSectionHeader"]})[0].text == 'Titles'):
                flag = 1

    else:
        print('Введеное название фильма не найдено.') 
        return 0 #TODO проработать возврат
    if flag == 1:
        href = soup.findAll("td", attrs={"class": ["result_text"]})[0].find('a')['href'] #ссылка на искомый фильм
        title = soup.findAll("td", attrs={"class": ["result_text"]})[0].find('a').text #название фильма
        #взять рейтинг на главной странице
        url_main = 'https://www.imdb.com' + href
        r = requests.get(url_main)
        soup = BeautifulSoup(r.text, "html.parser")
        if not soup.findAll("span", attrs={"itemprop": ["ratingValue"]}):
            return 0
        rating = float(soup.findAll("span", attrs={"itemprop": ["ratingValue"]})[0].text) 
        assessment = 0 if rating < 7.5 else 1
        print(url)
        #получить комментарии
        position_of_question = href.find('?')
        new_href = href[:position_of_question]
        url_reviews = 'https://www.imdb.com' + new_href + 'reviews?spoiler=hide&sort=helpfulnessScore&dir=desc&ratingFilter=0' #ссылка на reviews
        r = requests.get(url_reviews)
        soup = BeautifulSoup(r.text, "html.parser")      
        reviews = []

        #брать все отзывы, но обрезать. 5 отзывов по 20 слов

        length = 5 if len(soup.findAll("div", attrs={"class": ["content"]})) > 5 else len(soup.findAll("div", attrs={"class": ["content"]}))
        
        for i in range(length):
            review = soup.findAll("div", attrs={"class": ["content"]})[i].find('div').text
            review = review.replace('\n', ' ').replace('\r', '').replace('\t', '').replace(u'\xa0', u' ').replace('\x97', '—').replace('\x85', '...').replace('\x88', '^').replace("\'", "'")
            index = 0
            review = review[::-1]
            if len(review) > 60:             
                for i in range(60):
                    index = review.find(' ', index+1)
                review = review[:index] #обрезаем по 60 слов
            review = review[::-1]
            reviews.append(review)
        
        res = ''
        for i in reviews: #запихиваю все отзывы в 1 строку
            res = res + i + ' '
        return [[film_name, res,rating, assessment]]
    if 1 == 1:
        return 0


In [12]:
films = ['ghf']

In [5]:
with Pool(10) as p:
    records = p.map(parse, cars_links)

In [6]:
records

[]

In [1]:
items = [1, 2, 3, 4, 5]
squared = list(map(lambda x: x**2, items))

In [2]:
squared

[1, 4, 9, 16, 25]

In [ ]:
p = Pool(10)  # Pool tells how many at a time
records = p.map(parsing_reviews, films)
p.terminate()
p.join()